In [ ]:
import torch
import random 
import numpy as np
import datasets.addition_separate as dataset_manager

device="cuda"

In [ ]:
torch.cuda.is_available()

In [ ]:
random.seed(42)
np.random.seed(42)
sequence_length = 10

# Generate test-set of size 1000
test_Xs,test_Ys = dataset_manager.generate_dataset(1000,sequence_length)

In [ ]:
print(test_Xs[0][0])
print(test_Ys[0][0])

In [ ]:
from transformer import Transformer

model = Transformer(device,dataset_manager.vocab_len_in,dataset_manager.vocab_len_out, sequence_length)
model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
for epoch in range(100000):
    X_b,Y_b = dataset_manager.generate_dataset(1000,sequence_length)
    X_b = X_b.to(device)
    Y_b = Y_b.to(device)
    optimizer.zero_grad()
    out = model(X_b.T, Y_b.T)
    loss = model.loss(out, Y_b.T)
    loss.backward()
    optimizer.step()

    if epoch%10==0:
        # Evaluate on test set
        tX = test_Xs[:]
        tY = test_Ys[:]
        res = model(tX.T.to(device),tY.T.to(device)).argmax(dim=2).cpu().T
        not_end_token = tY[:,1:]!=dataset_manager.end_token_out
        digit_accuracy = torch.sum((res[:,:-1]==tY[:,1:]) & not_end_token) / torch.sum(not_end_token)

        print("Digit accuracy:", digit_accuracy, ", training_loss:", loss.detach().cpu().numpy())

        torch.save(model,"model")